In [4]:
import sys
print(sys.version)

3.7.1 (default, Oct 28 2018, 08:39:03) [MSC v.1912 64 bit (AMD64)]


In [5]:
import gym
print(gym.__version__)

0.18.0


In [6]:
import keras
print(keras.__version__)

Using TensorFlow backend.


2.3.1


In [7]:
import  random 
import numpy as np 
import math
from collections import deque


In [5]:
#seting up openai gym environment

env= gym.make('CartPole-v0')
for i_episode in range(20):
    observation =env.reset()
    for t in range(1000):
        env.render()
        #print observation
        action=env.action_space.sample()
        observation,reward,done,info=env.step(action)
        
        if done:
            print('Episode finished after {} timesteps'.format(t+1))
            break
            

Episode finished after 22 timesteps
Episode finished after 16 timesteps
Episode finished after 18 timesteps
Episode finished after 24 timesteps
Episode finished after 34 timesteps
Episode finished after 23 timesteps
Episode finished after 15 timesteps
Episode finished after 34 timesteps
Episode finished after 19 timesteps
Episode finished after 8 timesteps
Episode finished after 9 timesteps
Episode finished after 28 timesteps
Episode finished after 12 timesteps
Episode finished after 17 timesteps
Episode finished after 23 timesteps
Episode finished after 22 timesteps
Episode finished after 11 timesteps
Episode finished after 13 timesteps
Episode finished after 39 timesteps
Episode finished after 41 timesteps


In [6]:
print(env.action_space)
      
print(env.observation_space)

Discrete(2)
Box(-3.4028234663852886e+38, 3.4028234663852886e+38, (4,), float32)


In [7]:
print(env.observation_space.high)
print(env.observation_space.low)

[4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]
[-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38]


In [8]:
n_episode = 1000
n_win_ticks = 195
#timestamp is called tick 
max_env_steps = None

gamma = 1.0 #discount factor -future reward
epsilon = 1.0 # exploration random walk
epsilon_min = 0.01 
epsilon_decay  = 0.995
alpha = 0.01 #learnning rate
alpha_decay = 0.01

batch_size = 64
monitor =False
quiet= False#for controling  print statement

memory = deque (maxlen=100000)
env= gym.make('CartPole-v0')
if max_env_steps is not None: env.max_episode_steps= max_env_steps

In [9]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [10]:
model = Sequential()
model.add(Dense(24,input_dim= 4,activation='relu'))

model.add(Dense(48,activation='relu'))

model.add(Dense(2,activation='relu'))

model.compile(loss='mse',optimizer = Adam(lr= alpha, decay = alpha_decay) )

In [11]:
def remember(state,action,reward,next_state,done):
    memory.append((state,action,reward,next_state,done))

def choose_action(state,epsilon):
    return env.action_space.sample() if (np.random.random()<= epsilon) else np.argmax(model.predict(state))

def get_epsilon(t):
    return max(epsilon_min , min(epsilon ,1.0-math.log10(t+1)*epsilon_decay))

def preprocess_state(state):
    return  np.reshape(state,[1,4])

def replay(batch_size, epsilon):
    x_batch,y_batch=[],[]
    minibatch= random.sample(memory,min(len(memory),batch_size))
    for state,action,reward,next_state,done in minibatch:
        y_target=model.predict(state)
        y_target[0][action]=reward if done else reward + gamma * np.max(model.predict(next_state)[0])
        x_batch.append(state[0])
        y_batch.append(y_target[0])
        
    model.fit(np.array(x_batch),np.array(y_batch),batch_size=len(x_batch),verbose=0)
    if(epsilon>epsilon_decay):
        epsilon *= epsilon_decay

In [12]:
#run function 
def run():
    scores=deque(maxlen=100)
    
    for e in range(n_episode):
        state = preprocess_state(env.reset())
        done = False
        i =0
        while not done:
            action=choose_action(state,get_epsilon(e))
            next_state,reward,done,_=env.step(action)
            env.render()
            next_state=preprocess_state(next_state)
            remember(state,action,reward,next_state,done)
            state=next_state
            i+=1
            
        scores.append(i)
        mean_score=np.mean(scores)
        
        if mean_score>=n_win_ticks and e>=100:
            if not quiet : print('Ran {} episodes Solved after {} trials'.format(e,e-100))    
            return e-100 
            if (e%20 == 0 and not quiet):
                print('[Episode{}] - mean survivial time over last 20 episodes was {} ticks'.format(e,mean_score))
            
        replay(batch_size,get_epsilon(e))
        
    if not quiet :print('did not solved after {} episodes'.format(e))
    return e
            
        
            
        

In [ ]:
run()

In [ ]:
import random
import gym
import math
import numpy as np
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Training Parameters
n_episodes=1000
n_win_ticks=195
max_env_steps=None
gamma=1.0
epsilon=1.0
epsilon_min=0.01
epsilon_decay=0.995
alpha=0.01
alpha_decay=0.01
batch_size=64
monitor=False
quiet=False

# Environment Parameters
memory = deque(maxlen=100000)
env = gym.make('CartPole-v0')
if max_env_steps is not None: env.max_episode_steps = max_env_steps



# Model Definition
model = Sequential()
model.add(Dense(24, input_dim=4, activation='relu'))
model.add(Dense(48, activation='relu'))
model.add(Dense(2, activation='relu'))
model.compile(loss='mse', optimizer=Adam(lr=alpha, decay=alpha_decay))

def remember(state, action, reward, next_state, done):
    memory.append((state, action, reward, next_state, done))

def choose_action(state, epsilon):
    return env.action_space.sample() if (np.random.random() <= epsilon) else np.argmax(
        model.predict(state))

def get_epsilon(t):
    return max(epsilon_min, min(epsilon, 1.0 - math.log10((t + 1) * epsilon_decay)))

def preprocess_state(state):
    return np.reshape(state, [1, 4])

def replay(batch_size, epsilon):
    x_batch, y_batch = [], []
    minibatch = random.sample(
        memory, min(len(memory), batch_size))
    for state, action, reward, next_state, done in minibatch:
        y_target = model.predict(state)
        y_target[0][action] = reward if done else reward + gamma * np.max(model.predict(next_state)[0])
        x_batch.append(state[0])
        y_batch.append(y_target[0])

    model.fit(np.array(x_batch), np.array(y_batch), batch_size=len(x_batch), verbose=0)
    
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay    
        
def run():
    scores = deque(maxlen=100)

    for e in range(n_episodes):
        state = preprocess_state(env.reset())
        done = False
        i = 0
        while not done:
            action = choose_action(state, get_epsilon(e))
            next_state, reward, done, _ = env.step(action)
            env.render()
            next_state = preprocess_state(next_state)
            remember(state, action, reward, next_state, done)
            state = next_state
            i += 1

        scores.append(i)
        mean_score = np.mean(scores)
        if mean_score >= n_win_ticks and e >= 100:
            if not quiet: print('Ran {} episodes. Solved after {} trials'.format(e, e - 100))
            return e - 100
        if e % 20 == 0 and not quiet:
            print('[Episode {}] - Mean survival time over last 20 episodes was {} ticks.'.format(e, mean_score))
   
        replay(batch_size, get_epsilon(e))

    if not quiet: print('Did not solve after {} episodes'.format(e))
    return e

run()

[Episode 0] - Mean survival time over last 20 episodes was 22.0 ticks.


Exception ignored in: <function Viewer.__del__ at 0x000001D7868106A8>
Traceback (most recent call last):
  File "c:\users\91750\desktop\dl\gym\gym\envs\classic_control\rendering.py", line 165, in __del__
    self.close()
  File "c:\users\91750\desktop\dl\gym\gym\envs\classic_control\rendering.py", line 83, in close
    self.window.close()
  File "C:\Users\91750\Anaconda\lib\site-packages\pyglet\window\win32\__init__.py", line 299, in close
    super(Win32Window, self).close()
  File "C:\Users\91750\Anaconda\lib\site-packages\pyglet\window\__init__.py", line 823, in close
    app.windows.remove(self)
  File "C:\Users\91750\Anaconda\lib\_weakrefset.py", line 109, in remove
    self.data.remove(ref(item))
KeyError: (<weakref at 0x000001D789285228; to 'Win32Window' at 0x000001D7FD838550>,)


[Episode 20] - Mean survival time over last 20 episodes was 11.095238095238095 ticks.
